In [ ]:
!git clone https://github.com/sushant097/TSAI-ERAv1-Assignments.git

In [ ]:
%cd TSAI-ERAv1-Assignments/subModules/VAE

In [ ]:
!pip install -r requirements.txt --quiet


In [ ]:
!pip install torchsummary --quiet

In [ ]:
! pip install git+https://github.com/PytorchLightning/lightning-bolts.git@master --upgrade --quiet


In [ ]:
import torch
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from config import *
from lightning_model.VAELitModel import VAELitModel
from lightning_datamodule.mnist import MNISTDataModule

In [ ]:
datamodule = MNISTDataModule(
    data_dir=DATA_DIR,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)
vae = VAELitModel()

trainer = pl.Trainer(
    gpus=1,
    max_epochs=8
)
trainer.fit(vae, datamodule)

In [ ]:

test_loader = datamodule.test_dataloader()

dataiter = iter(test_loader)
data_next = next(dataiter)
images, labels = data_next
     

In [ ]:

incorrect_labels = []

for i in labels:
    incorrect_label = torch.randint(low=0, high=10, size=(1,)).item()

    while incorrect_label == i:
        incorrect_label = torch.randint(low=0, high=10, size=(1,)).item()

    incorrect_labels.append(incorrect_label)

In [ ]:
device = "cpu"

labels_corr = torch.IntTensor(incorrect_labels)
images, labels_corr = images.to(device), labels_corr.to(device)

with torch.no_grad():
    pred = vae((images, labels_corr))

In [ ]:

ind = 0
fig = plt.figure(figsize=(15,15))
for i in range(25):
    label_corr = labels[i]
    img_corr = images[i]
    img_new = pred[i]
    label_new = labels_corr[i]

    ind = ind+1
    ax = fig.add_subplot(10, 10,ind )
    ax.axis("off")
    ax.imshow(np.transpose(img_corr.squeeze(0).cpu().numpy(), (1, 2, 0)), cmap='gray')
    ax.set_title(f'\n Incorrect Label: {label_new.detach()}',fontsize=10)

    ind = ind+1
    ax = fig.add_subplot(10, 10, ind)
    ax.axis("off")
    ax.imshow(np.transpose(img_new.squeeze(0).cpu().detach().numpy(), (1, 2, 0)), cmap='gray')